## Overview

The Keras Tuner is a library that helps you pick the optimal set of hyperparameters for your TensorFlow program. The process of selecting the right set of hyperparameters for your machine learning (ML) application is called *hyperparameter tuning* or *hypertuning*.

Hyperparameters are the variables that govern the training process and the topology of an ML model. These variables remain constant over the training process and directly impact the performance of your ML program. Hyperparameters are of two types:
1. **Model hyperparameters** which influence model selection such as the number and width of hidden layers
2. **Algorithm hyperparameters** which influence the speed and quality of the learning algorithm such as the learning rate for Stochastic Gradient Descent (SGD) and the number of nearest neighbors for a k Nearest Neighbors (KNN) classifier

In this tutorial, you will use the Keras Tuner to perform hypertuning for an image classification application.

## Setup

In [ ]:
import tensorflow as tf
from tensorflow import keras

Install and import the Keras Tuner.

In [ ]:
!pip install -q -U keras-tuner

In [ ]:
import keras_tuner as kt

## Download and prepare the dataset

In this tutorial, you will use the Keras Tuner to find the best hyperparameters for a machine learning model that classifies images of clothing from the [Fashion MNIST dataset](https://github.com/zalandoresearch/fashion-mnist).

Load the data.

fashion mnist consists of 70000 images, 60000 images are in train and the remaining 10000 images are in test.

size is 28 x 28 px

In [ ]:
#import fashion mnist data into 4 arrays
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

All images are loaded in the form of array of pixel values in range of 0 to 255.

In [ ]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

## Define the model

When you build a model for hypertuning, you also define the hyperparameter search space in addition to the model architecture. The model you set up for hypertuning is called a *hypermodel*.

You can define a hypermodel through two approaches:

* By using a model builder function (customised)
* By subclassing the `HyperModel` class of the Keras Tuner API

You can also use two pre-defined `HyperModel` classes - [HyperXception](https://keras-team.github.io/keras-tuner/documentation/hypermodels/#hyperxception-class) and [HyperResNet](https://keras-team.github.io/keras-tuner/documentation/hypermodels/#hyperresnet-class) for computer vision applications.

In this tutorial, you use a model builder function to define the image classification model. The model builder function returns a compiled model and uses hyperparameters you define inline to hypertune the model.

In [ ]:
from keras import layers
from keras.layers import BatchNormalization
from keras import regularizers

def model_builder(hp):#declare our ANN architecture: input, hidden and output layer
  model = keras.Sequential()#instantiating the sequential method
  #adding input layer
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512)
  model.add(keras.layers.Dense(units=hp_units, activation='relu',kernel_regularizer=regularizers.l2(0.01)))
  model.add(BatchNormalization())
  model.add(layers.Dropout(0.5))

  model.add(keras.layers.Dense(10)) #output layer

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

In [ ]:
from keras import layers
from keras.layers import BatchNormalization
from keras import regularizers

def model_builder(hp):#declare our ANN architecture: input, hidden and output layer
  model = keras.Sequential()#instantiating the sequential method
  #adding input layer
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512)
  model.add(keras.layers.Dense(units=hp_units, activation='relu',kernel_regularizer=regularizers.l2(0.01)))

  model.add(keras.layers.Dense(10)) #output layer

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

In [ ]:
from keras_tuner import HyperModel
class ANNHyperModel(HyperModel):
  def build(self, hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28)))
    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 32-512
    hp_units = hp.Int('units', min_value=32, max_value=512)
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))

    model.add(keras.layers.Dense(10)) #output layer

    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

    return model

hypermodel = ANNHyperModel()



## Instantiate the tuner and perform hypertuning

Instantiate the tuner to perform the hypertuning. The Keras Tuner has four tuners available - `RandomSearch`, `Hyperband`, `BayesianOptimization`, and `Sklearn`. In this tutorial, you use the [Hyperband](https://arxiv.org/pdf/1603.06560.pdf) tuner.

To instantiate the Hyperband tuner, you must specify the hypermodel, the `objective` to optimize and the maximum number of epochs to train (`max_epochs`) one model

hypermodel is a model builder function we defined previously;

project_name is a path to the folder where all tuning-related results will be placed and stored.

In [ ]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

In [ ]:
tuner = kt.Hyperband(hypermodel,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

The Hyperband tuning algorithm uses adaptive resource allocation and early-stopping to quickly converge on a high-performing model.

Create a callback to stop training early after reaching a certain value for the validation loss.

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

Run the hyperparameter search. The arguments for the search method are the same as those used for `tf.keras.model.fit` in addition to the callback above.

In [ ]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

In [ ]:
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
#number of units in the hidden layer
#learning rate

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

#num_trials: Optional number of HyperParameters objects to return.

#Returns
#List of HyperParameter objects sorted from the best to the worst.


The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 349 and the optimal learning rate for the optimizer
is 0.0001.



## Train the model

Find the optimal number of epochs to train the model with the hyperparameters obtained from the search.

In [ ]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
newmodel = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = newmodel.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 10s 6ms/step - loss: 4.1733 - accuracy: 0.7496 - val_loss: 2.9080 - val_accuracy: 0.8384
Epoch 2/50
1500/1500 [==============================] - 5s 3ms/step - loss: 2.3916 - accuracy: 0.8152 - val_loss: 1.8106 - val_accuracy: 0.8523
Epoch 3/50
1500/1500 [==============================] - 6s 4ms/step - loss: 1.5641 - accuracy: 0.8319 - val_loss: 1.2377 - val_accuracy: 0.8577
Epoch 4/50
1500/1500 [==============================] - 5s 3ms/step - loss: 1.1252 - accuracy: 0.8376 - val_loss: 0.9247 - val_accuracy: 0.8589
Epoch 5/50
1500/1500 [==============================] - 6s 4ms/step - loss: 0.8806 - accuracy: 0.8445 - val_loss: 0.7618 - val_accuracy: 0.8575
Epoch 6/50
1500/1500 [==============================] - 5s 4ms/step - loss: 0.7388 - accuracy: 0.8487 - val_loss: 0.6314 - val_accuracy: 0.8673
Epoch 7/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.6499 - accuracy: 0.8494 - val_loss: 0.6165 - val_accuracy

Re-instantiate the hypermodel and train it with the optimal number of epochs from above.

In [ ]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/28
1500/1500 [==============================] - 6s 3ms/step - loss: 4.0914 - accuracy: 0.7472 - val_loss: 2.8226 - val_accuracy: 0.8387
Epoch 2/28
1500/1500 [==============================] - 6s 4ms/step - loss: 2.3137 - accuracy: 0.8189 - val_loss: 1.7461 - val_accuracy: 0.8559
Epoch 3/28
1500/1500 [==============================] - 5s 3ms/step - loss: 1.5355 - accuracy: 0.8309 - val_loss: 1.2241 - val_accuracy: 0.8540
Epoch 4/28
1500/1500 [==============================] - 6s 4ms/step - loss: 1.1132 - accuracy: 0.8390 - val_loss: 0.9510 - val_accuracy: 0.8490
Epoch 5/28
1500/1500 [==============================] - 5s 3ms/step - loss: 0.8797 - accuracy: 0.8440 - val_loss: 0.7483 - val_accuracy: 0.8624
Epoch 6/28
1500/1500 [==============================] - 6s 4ms/step - loss: 0.7422 - accuracy: 0.8464 - val_loss: 0.6704 - val_accuracy: 0.8543
Epoch 7/28
1500/1500 [==============================] - 6s 4ms/step - loss: 0.6484 - accuracy: 0.8515 - val_loss: 0.5697 - val_accuracy:

To finish this tutorial, evaluate the hypermodel on the test data.

In [ ]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

313/313 [==============================] - 1s 3ms/step - loss: 0.4398 - accuracy: 0.8679
[test loss, test accuracy]: [0.4397989809513092, 0.867900013923645]


The `my_dir/intro_to_kt` directory contains detailed logs and checkpoints for every trial (model configuration) run during the hyperparameter search. If you re-run the hyperparameter search, the Keras Tuner uses the existing state from these logs to resume the search. To disable this behavior, pass an additional `overwrite=True` argument while instantiating the tuner.